# Byte-at-a-time ECB decryption (Harder)
Take your oracle function from #12. Now generate a random count of random bytes and prepend this string to every plaintext. You are now doing:

```
AES-128-ECB(random-prefix || attacker-controlled || target-bytes, random-key)
```

Same goal: decrypt the target-bytes.

### Stop and think for a second.
> What's harder than challenge #12 about doing this? How would you overcome that obstacle? The hint is: you're using all the tools you already have; no crazy math is required.
>
> Think "STIMULUS" and "RESPONSE".


In [1]:
from importlib import reload
import cryptopals
reload(cryptopals)
from cryptopals import *

In [229]:
def prepending_ECB_oracle(plaintext, prefix, key):
    '''Oracle that appends a mystery string to the plaintext input before encrypting via ECB.'''
    unknown_string = base64_to_bytes('Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK')
    plaintext = pad_pkcs7(prefix + to_bytes(plaintext) + unknown_string)
    
    return AES.new(key, AES.MODE_ECB).encrypt(plaintext)

def find_unknown_string_from_prepending_oracle():
    # generate consistent but unknown prefix
    consistent_random_prefix = generate_random_bytes(np.random.randint(AES.block_size*10)+1)
    # generate consistent but unknown key
    consistent_unknown_key = generate_random_bytes()
    
    def oracle(plaintext):
        return prepending_ECB_oracle(plaintext, consistent_random_prefix, consistent_unknown_key)
    
    # find prefix length
    start_buffer_len = 0
    while True:
        ct = oracle(chr(0)*start_buffer_len)
        blocks = [ct[i*AES.block_size : (i+1)*AES.block_size] for i in range(len(ct) // AES.block_size)]
        
        if max(Counter(blocks).values()) == 2:
            break
        
        start_buffer_len += 1

    # find unknown message, one byte at a time
    start_idx = ct.find(Counter(blocks).most_common(1)[0][0]) + AES.block_size*2
    hidden_string = b''
    
    while True:
        idx_next_letter = len(hidden_string) + start_idx
        idx_start_of_block = idx_next_letter - idx_next_letter%AES.block_size
        short_pt = chr(0)*(AES.block_size-1-idx_next_letter%AES.block_size)
        
        ct_block_to_char_map = {}
        for i in range(256):
            # note that chr(128)..chr(255) do not properly convert to bytes via .encode()
            pt = (chr(0)*start_buffer_len + short_pt).encode() + hidden_string + bytes([i])
            ct = oracle(pt)
            unknown_block = ct[idx_start_of_block:idx_start_of_block+AES.block_size]
            ct_block_to_char_map[unknown_block] = bytes([i])

        ct = oracle(chr(0)*start_buffer_len + short_pt)
        unknown_block = ct[idx_start_of_block:idx_start_of_block+AES.block_size]
        if unknown_block not in ct_block_to_char_map:
            # signals end of message because padding bytes don't match any possible
            return hidden_string[:-1].decode()
        hidden_string += ct_block_to_char_map[unknown_block]
        

In [24]:
consistent_random_prefix = generate_random_bytes(np.random.randint(AES.block_size*10)+1)

In [7]:
consistent_unknown_key = generate_random_bytes()

In [197]:
for i in range(10000):
    pt = chr(0)*i
    ct = prepending_ECB_oracle(pt, consistent_random_prefix, consistent_unknown_key)
    blocks = [ct[i*AES.block_size : (i+1)*AES.block_size] for i in range(len(ct) // AES.block_size)]
    
    if max(Counter(blocks).values()) == 2:
        break
        
start_buffer_len = i
print(start_buffer_len)

47


In [198]:
start_idx = ct.find(Counter(blocks).most_common(1)[0][0]) + AES.block_size*2
hidden_string = b''

In [199]:
Counter(blocks).most_common(1)[0][0]

b'7\xe8\x90\x1c\x81\xcc=\xea\xf7u\x96\xc3\xbf\x1d\xc9\t'

In [200]:
ct[:start_idx], ct[start_idx:]

(b'\xc97\x82\x84\xfb\x94\xc7k\x97C\xe1O\xc0\x0e\xf6\x17\x1b\xafG\xa1\xbeN\x88\xb0\x1bt}\x16\xb1p\xd1p\xdcB;\xe8P\xbe\xd5\xbd\x99qrQ\xf4\r\xce\x93\xa4\xbc\x0f\x98\xc46\x1f0V\x008\r\xd8/\xfe\xb1\x82\xe2OC\xb0\xbb\xbd\x8ca\xb8\x8c(RID<7\xe8\x90\x1c\x81\xcc=\xea\xf7u\x96\xc3\xbf\x1d\xc9\t7\xe8\x90\x1c\x81\xcc=\xea\xf7u\x96\xc3\xbf\x1d\xc9\t',
 b'\xca\x98{G%\xad\x16\x0e\x81))\xdc\xae\xdaSUo\xe4&\x1dfl\xb8\t\xb5\x87\x88D\xfb\xacw\x98\x08\xbe\xc4\tK\xe7X\xae!c#\xf5U\x055\x95\xb6)\xdcf\xb1\x16\xd9\x86[\xb6\x19H\xe8\xf1"\xa0\xc8\xe1\xa6b\x13f\x05\xc1j\xee!%\xb4\xbc\x0c\x81\xa8\xbf\x82B\xdd^t\xfelS\x8d\x02\x1b\xb1\xf0\xd0\xae\xd1\xf2\x80\x02\x8f"\xac\xb7\xddE\x7fpX\xdb\x82\x97\xba\x17\x80|\xbb\x8f\xa8\xa9\xda:\xef\x07c\x82\x94\x17)g\x88@]2\xe7\xeb\xbe\xb1\xf0\xc9y\xdeT')

In [217]:
idx_next_letter = len(hidden_string) + start_idx
idx_start_of_block = idx_next_letter - idx_next_letter%AES.block_size
short_pt = chr(0)*(AES.block_size-1-idx_next_letter%AES.block_size)

In [218]:
ct_block_to_char_map = {}
for i in range(256):
    # note that chr(128)..chr(255) do not properly convert to bytes via .encode()
    pt = (chr(0)*start_buffer_len + short_pt).encode() + hidden_string + bytes([i])
    ct = prepending_ECB_oracle(pt, consistent_random_prefix, consistent_unknown_key)
    unknown_block = ct[idx_start_of_block:idx_start_of_block+AES.block_size]
    ct_block_to_char_map[unknown_block] = bytes([i])
#     print(unknown_block, bytes([i]))

In [219]:
ct = prepending_ECB_oracle(chr(0)*start_buffer_len + short_pt, consistent_random_prefix, consistent_unknown_key)
unknown_block = ct[idx_start_of_block:idx_start_of_block+AES.block_size]
if unknown_block not in ct_block_to_char_map:
    # signals end of message because padding bytes don't match any possible
    print('No soln')
hidden_string += ct_block_to_char_map[unknown_block]
print(hidden_string)

b'Roll'


In [230]:
%%time
print(find_unknown_string_from_prepending_oracle())

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by

CPU times: user 423 ms, sys: 4.72 ms, total: 428 ms
Wall time: 435 ms
